# 3. Основы синтасксиса: Массивы


## 3.1. Array

В Julia для массивов существует абстрактный тип `AbstractArray{T,N}`, где `T` тип элемента массива (`Int64`, `String`, `Complex{Int64}`, `Number`, `Any`...), а `N` размерность. При написании собственных типов с поведением массива в объявлении следует указать, что ваш тип является подтипом `AbstractArray`.

Встроенным типом для массивом является `Array{T,N}`. Это изменяемый массив динамического размера с *column-major* хранением элементов.

Синтаксис: <br>
```julia
[item1, item2, ...]
```

>`NOTE` Поскольку в Julia массивы хранятся постолбично, то для быстродействия вложенный цикл должен проходится по строке, а внешний по столбцу. Вообще, если есть массив `A[i, j, k]`, то цикл должен выглядить так:
>
>```julia
>for k in eachindex(A, 3)
>    for j in eachindex(A, 2)
>        for i in eachindex(A, 1)
>            ...
>        end
>    end
>end
>```

Существуют два часто используемых синонима

- `Vector{T} == Array{T,1}` для одномерных вектор-столбцов;
- `Matrix{T} == Array{T,2}` для матриц и вектор-строк.

Литералом `Array{T,N}` являются квадратные скобки `[]`, при этом можно явно указать тип `T[]` (`Int[1, 2]`, `Float64[1, 2]`).
Если тип элементов массива не указывать, то Julia его вычислит.

Для создания массива через литерал используется вертикальная и горизонтальная *конкатенация*.

- Для **вертикальной конкатенации** используется `;` или новая строка.
 
- Для **горизонтальной конкатенации** используется пробел ` ` или табуляция. 

Запятая `,` означает отсутствие конкатенации. Если конкатенация не нужна, используйтся запятая `,`. В примере ниже показано создание вектора векторов.

In [62]:
@show a = [1, 2, 3] typeof(a) # Вектор из скаляров
@show b = [[1, 2, 3]; [7, 8]] typeof(b) # Вертикальная конкатенация

@show a2 = [1 2 3 4] typeof(a2) # Вектор-строка
@show b2 = [[1, 2] [3, 4]] typeof(b2)# Горизонтальная конкатенация

@show b3 = [[1, 2, 3], [7, 8]] typeof(b3); # Не конкатенация



a = [1, 2, 3] = [1, 2, 3]
typeof(a) = Vector{Int64}
b = [[1, 2, 3]; [7, 8]] = [1, 2, 3, 7, 8]
typeof(b) = Vector{Int64}
a2 = [1 2 3 4] = [1 2 3 4]
typeof(a2) = Matrix{Int64}
b2 = [[1, 2] [3, 4]] = [1 3; 2 4]
typeof(b2) = Matrix{Int64}
b3 = [[1, 2, 3], [7, 8]] = [[1, 2, 3], [7, 8]]
typeof(b3) = Vector{Vector{Int64}}


На первый взгляд синтаксис может показаться запутанным.
Однако, с ним легко создавать блочные матрицы и тензоры.

Скажем, задана блочная матрица

$
\mathcal{M} = \left[\begin{array}{ccc|ccc}
1 & 2 & 3 & 1 & 0 & 0 \\
4 & 5 & 6 & 0 & 1 & 0 \\
7 & 8 & 9 & 0 & 0 & 1 \\ \hline
0 & 0 & 0 & 10 & 11 & 12 \\
\end{array}\right]
= \left[\begin{array}{c|c}
A & I \\ \hline
0^\top & b^\top
\end{array}\right]
$

Соответствующий код будет:


In [23]:
A = [1 2 3; 4 5 6; 7 8 9]   # или reshape(collect(1:9), 3, 3)'
I = [1 0 0; 0 1 0; 0 0 1]
zero = [0, 0, 0]  # или zeros(Int, 3)
b = [10, 11, 12]  # или collect(10:12)

M = [A I; zero' b']

4×6 Matrix{Int64}:
 1  2  3   1   0   0
 4  5  6   0   1   0
 7  8  9   0   0   1
 0  0  0  10  11  12

Оператор `'` (одинарная кавычка) выполняет `транспонирование` (cм. раздел 5.2).

**Индексация**

В Julia встроено два вида индексов.
При этом нужно помнить, что в Julia-массивах значения хранятся постолбично.

Синтаксис первого `A[i]`, это линейный индекс `LinearIndex`.
Он позволяет получить `i`-ый *по счёту в памяти* элемент массива.
Например, для вектора `A[4]` это $A_4$, а для матрицы размера $2 \times 3$ это $A_{2, 2}$.

Синтаксис второго `A[i, j, k,..]`, это декартов индекс `CartesianIndex`. Данный индекс учитывает размерность массива. Например, для матрицы элемент `A[i, j]` это элемент $A_{i,j}$ ($i$-ая строка, $j$-ый столбец).

Индексом может быть и коллекция элементов, что позволяет обращаться к целым строкам, столцам, их пересечению, или просто некоторому набору элементов. Часто для этого используется *range operator* `start:[step:]stop`.

In [155]:
A = [1 2 3; 
     4 5 6; 
     7 8 9]
@show A
@show A[8] # линейный индекс
@show A[2, end] # декартов индекс
@show A[2, :]  # A[строка 1, все столбцы]
@show A[:, 2]  # A[все строки, столбец 2]
@show A[1:2, 2:3] # "Пересечение" строк 1 и 2 со столбцами 2 и 3.
@show A[1:4:end] # Главная диагональ
@show A[3:2:7]; # Побочная диагональ

A = [1 2 3; 4 5 6; 7 8 9]
A[8] = 6
A[2, end] = 6
A[2, :] = [4, 5, 6]
A[:, 2] = [2, 5, 8]
A[1:2, 2:3] = [2 3; 5 6]
A[1:4:end] = [1, 5, 9]
A[3:2:7] = [7, 5, 3]


### Вспомогательные функции

In [184]:
@show vector = [1, 2, 3]
@show matrix = [1.0 2.0 4.0;
                5.0 6.0 7.0]

# Интроспекция массива
@show length(matrix) # количество элементов
@show ndims(matrix) # размерность
@show size(matrix) # кортеж из размеров каждой размерности

# Создание массивов
@show collect(0:2:10) # создание массива из коллекции;
@show zeros(Float64, (3,2)) # массив из нулей типа `Float64` размерности `(3,2)`
@show ones(Float64, 3) # аналогично `zeros`, но массив из единиц;
@show copy(matrix) # нерекурсивная копия массива
@show similar(matrix) # неинициализированный массив того же типа и размера, что и `matrix`
@show rand(Float16, (3,3)) # массив из равномерно-распределённых случайных чисел;
@show fill(2.0, (2,3)) # массив размера `(2,3)` из `2.0`;
@show map(sqrt, vector) # создание массива на основе применения функции `sin` к каждому элементу `vector`. 

# Другие операции
@show push!(vector, 100) # добавление в конец элемента
@show vector

@show pop!(vector) # извлечение элемента с конца
@show vector
# массив с различными типами данных
multi_types = [
    1, 1.0, 1//3, π,
    'a', "doggo", [7 8 9]
]
typeof(multi_types)

# Заполнение массива через цикл
@show cubed = [i^3 for i = 1:10];

vector = [1, 2, 3] = [1, 2, 3]
matrix = [1.0 2.0 4.0; 5.0 6.0 7.0] = [1.0 2.0 4.0; 5.0 6.0 7.0]
length(matrix) = 6
ndims(matrix) = 2
size(matrix) = (2, 3)
collect(0:2:10) = [0, 2, 4, 6, 8, 10]
zeros(Float64, (3, 2)) = [0.0 0.0; 0.0 0.0; 0.0 0.0]
ones(Float64, 3) = [1.0, 1.0, 1.0]
copy(matrix) = [1.0 2.0 4.0; 5.0 6.0 7.0]
similar(matrix) = [5.0e-324 1.5e-323 2.5e-323; 1.0e-323 2.0e-323 3.0e-323]
rand(Float16, (3, 3)) = Float16[0.0669 0.995 0.6553; 0.5884 0.1807 0.7695; 0.6797 0.6406 0.355]
fill(2.0, (2, 3)) = [2.0 2.0 2.0; 2.0 2.0 2.0]
map(sqrt, vector) = [1.0, 1.4142135623730951, 1.7320508075688772]
push!(vector, 100) = [1, 2, 3, 100]
vector = [1, 2, 3, 100]
pop!(vector) = 100
vector = [1, 2, 3]
cubed = [i ^ 3 for i = 1:10] = [1, 8, 27, 64, 125, 216, 343, 512, 729, 1000]


## 3.2. Tuple, Dict, Set

### Tuple

Мы можем создать кортеж, заключив упорядоченную коллекцию элементов в `( )`. Основное отличие от Array в том, что мы не можем изменять значения в кортежах

Синтаксис: <br>
```julia
(item1, item2, ...)
```

In [56]:
myfavoriteanimals = ("penguins", "cats", "sugargliders")
@show typeof(myfavoriteanimals)
@show myfavoriteanimals[1]
@debug myfavoriteanimals[1] = "otters"

typeof(myfavoriteanimals) = Tuple{String, String, String}
myfavoriteanimals[1] = "penguins"


Тип `NamedTuple` работает точно также как `Tuple` за исключением того, что каждый элемент дополнительно имеет имя. У них есть специальный синтаксис, использующий `=` внутри кортежа:

```julia
(name1 = item1, name2 = item2, ...)
```

In [60]:
myfavoriteanimals = (bird = "penguins", mammal = "cats", marsupial = "sugargliders")
@show myfavoriteanimals[1]
@show myfavoriteanimals.bird;

myfavoriteanimals[1] = "penguins"
myfavoriteanimals.bird = "penguins"


### Dict

Если у нас есть наборы данных, связанных друг с другом, мы можем сохранить эти данные в словаре. Мы можем создать словарь, используя функцию `Dict()`.

Синтаксис:
```julia
Dict(key1 => value1, key2 => value2, ...)
```

In [1]:
red = (255, 0, 0)
Jenny = Dict(
    "name" => "Jenny",
    "hair" => :red,
    :age => 13,
)

@show Jenny["name"]
Jenny[:email] = "jenny@mail.com"
@show Jenny
pop!(Jenny, :email)
@show Jenny;



Jenny["name"] = "Jenny"
Jenny = Dict{Any, Any}("name" => "Jenny", :age => 13, "hair" => :red, :email => "jenny@mail.com")
Jenny = Dict{Any, Any}("name" => "Jenny", :age => 13, "hair" => :red)


В данном примере используется особый тип даных Symbol, соответствующий именам переменных:


In [4]:
name = :egg
@show typeof(name)

egg = 1
@show name
@show eval(:egg); #вывод значения переменной

typeof(name) = Symbol
name = :egg
eval(:egg) = 1


### Set

Множество — это структура данных, эквивалентная множествам в математике. Множество может состоять из различных элементов, порядок элементов в множестве неопределен. В множество можно добавлять и удалять элементы, можно перебирать элементы множества, можно выполнять операции над множествами (объединение, пересечение, разность). Можно проверять принадлежность элемента множеству.

В отличие от массивов, где элементы хранятся в виде последовательного списка, в множествах порядок хранения элементов неопределен (более того, элементы множества хранятся не подряд, как в списке, а при помощи хитрых алгоритмов). Это позволяет выполнять операции типа “проверить принадлежность элемента множеству” быстрее, чем просто перебирая все элементы множества.

Элементами множества могут быть различные типы данных: числа, строки, кортежи. 

`Set([itr])` - Создаёт набор значений, сгенерированных данным итерируемым объектом, или пустой набор. Должен использоваться вместо BitSet для разреженных целочисленных наборов или для наборов произвольных объектов.

`BitSet([itr])` - Создаст отсортированный набор целочисленных объектов, сгенерированный данным итерируемым объектом, или пустой набор. Реализуется как битовая строка и, следовательно, предназначена для плотных целочисленных наборов. Если набор будет редким (например, содержит несколько очень больших целых чисел), используйте вместо него Set.

In [182]:
@show Set([1:6;])
@show Set([3,4,5,5,4,4,1,7,7]);
@show Set("abrakadabra")
@show Set([Set([1:6;]),Set("abrakadabra")])
@show Set((4,5,1,4,6,'a','q'))
@time Set([1000000:-1:1...])
@time BitSet([1000000:-1:1...]);

Set([1:6;]) = Set([5, 4, 6, 2, 3, 1])
Set([3, 4, 5, 5, 4, 4, 1, 7, 7]) = Set([5, 4, 7, 3, 1])
Set("abrakadabra") = Set(['a', 'd', 'r', 'k', 'b'])
Set([Set([1:6;]), Set("abrakadabra")]) = Set(Set[Set([5, 4, 6, 2, 3, 1]), Set(['a', 'd', 'r', 'k', 'b'])])
Set((4, 5, 1, 4, 6, 'a', 'q')) = Set(Any[5, 4, 6, 'a', 'q', 1])
  0.371014 seconds (3.00 M allocations: 118.118 MiB, 56.10% gc time)
  0.149196 seconds (3.00 M allocations: 100.436 MiB, 19.01% gc time)


Для работы с множествами существует ряд функций:

In [180]:
b = Set([3,4,5,5,4,4,1,7,7]);
@show b
@show length(b)
for b_i in b
    print("$b_i ")
end
print("\n")
@show b.*6
@show 5 in b
@show sqrt.(b);

b = Set([5, 4, 7, 3, 1])
length(b) = 5
5 4 7 3 1 
b .* 6 = [30, 24, 42, 18, 6]
5 in b = true
sqrt.(b) = [2.23606797749979, 2.0, 2.6457513110645907, 1.7320508075688772, 1.0]


А также специфические операции для множеств:

In [179]:
a = Set([1:6;])
b = Set([3,4,5,5,4,4,1,7,7])
@show union(a,b) # объединение
@show a ∪ b # \cup + TAB

@show intersect(a, b) # пересечение
@show a ∩ b # \cap + TAB

@show setdiff(a,b) # разность
@show symdiff(a,b) # симметрическая разность

@show issubset(a, [1, 2, 3]) # проверяет, входят ли элементы множества a в b  
@show [1,2,3] ⊆ a # \subseteq + TAB
@show [1,2,3] ⊇ a # \supseteq + TAB
@show (1, 2) ⊈ (2, 3) # отрицание 
@show (1, 2) ⊈ (1, 2, 3);


union(a, b) = Set([5, 7, 1, 4, 6, 2, 3])
a ∪ b = Set([5, 7, 1, 4, 6, 2, 3])
intersect(a, b) = Set([5, 4, 3, 1])
a ∩ b = Set([5, 4, 3, 1])
setdiff(a, b) = Set([6, 2])
symdiff(a, b) = Set([6, 7, 2])
issubset(a, [1, 2, 3]) = false
[1, 2, 3] ⊆ a = true
[1, 2, 3] ⊇ a = false
(1, 2) ⊈ (2, 3) = true
(1, 2) ⊈ (1, 2, 3) = false
